# Практика

In [ ]:
import sys
sys.path.append('/Users/kbashevoy/Desktop/Нетология/Занятия/Занятие 8')

from analytics_lib import *  # НИКОГДА так не делайте

In [ ]:
import request

In [ ]:
def useful_func(x, y, z):
    """Просто функции внутри классов - НЕ используют атрибуты"""
    return x**2 + y**2 + z**2

In [ ]:
def price_vat():
    ...

In [ ]:
self.cache = {
    'params': 'result'
}

In [ ]:
def func(a, b, c):

In [58]:
hash((1, 2, 3))

529344067295497451

In [ ]:
cache:
    '529344067295497451': 923,

In [ ]:
func(1, 2, 3) --> 

In [56]:
class AnyClass:  # CamelCase snake_case
    def __init__(self, currency='BLR', check=False):
        print('Выполняется при обращении к классу')
        # создаем все атрибуты, которые нужны для работы
        self.currency = currency
        
        self._api_version = 'v5.31'  # рекомендация НЕ менять значение вне кода класса
        self.__secret_attr = '123'  # запрет на изменение вне кода класса
        
        self.check = check
        
        self.price = 100
        self.discount = 0.1
    
    # функции внутри класса - методы
    def method_1(self):
        # currency = 'RUB'  # ничем не отличается от переменной функции
        self.currency = 'RUB'  # атрибут
        
        self._api_version = 'v5.31'  # рекомендация НЕ менять значение вне кода класса
        self.__secret_attr = '123'  # запрет на изменение вне кода класса
        
    def method_2(self):
        print(self.currency, self.__secret_attr)
        
    @staticmethod  # декоратор
    def useful_func(x, y, z):
        """Просто функции внутри классов - НЕ используют атрибуты"""
        return x**2 + y**2 + z**2
    
    @property
    def price_vat_discount(self):
        """Вычисляемый атрибут"""
        price_vat = self.price * 1.2
        
        return price_vat * (1 - self.discount)

In [57]:
a = AnyClass()

Выполняется при обращении к классу


In [52]:
a.price_vat_discount

108.0

In [47]:
a.useful_func(1, 2, 3)

TypeError: useful_func() takes 3 positional arguments but 4 were given

In [44]:
AnyClass().useful_func(1, 2, 3)

Выполняется при обращении к классу


14

In [40]:
a.method_2()

BLR 123


In [25]:
a.method_1()

In [22]:
a.__secret_attr

AttributeError: 'AnyClass' object has no attribute '__secret_attr'

In [18]:
a._api_version = 'v.6.0'
a._api_version

'v.6.0'

In [12]:
a.currency = 'EUR'
a.currency

'EUR'

In [13]:
a.method_2()

EUR


In [1]:
def func_1():
    x = 1 
    return x + 1

In [5]:
def func_2(x):
    print(x)

In [6]:
result = func_1()

In [7]:
func_2(result)

2


# Требования к отчету
(в хронологическом порядке)

1. Нужен отчет по трафику (визиты по дням)
2. Сделайте таблицу попроще
3. А можно сразу очень много строк получить?

[Демо-счетчик](https://metrika.yandex.ru/stat/traffic?sort=-ym%3As%3Avisits&period=2020-08-01%3A2020-09-06&accuracy=1&id=44147844&stateHash=5d3ee51607d6bde783131e11) Яндекс.Метрики


In [59]:
import requests

In [60]:
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
METRIKA_ROWS_LIMIT = 5  # max 100000

DEMO_COUNTER = 44147844

In [80]:
class Metrika:
    def __init__(self, token='', counter=DEMO_COUNTER, start_date='2020-09-01', end_date='2020-09-10'):
        self.counter = counter
        self.limit = METRIKA_ROWS_LIMIT
        self.start_date = start_date
        self.end_date = end_date
        
        if token:
            self.headers = {'Authorization': 'OAuth ' + token}
        else:
            self.headers = ''  # для демо-счетчика заголовок запроса должен быть пустым
        
        self.dimensions = ''  # строки отчета
        self.metrics = ''  # столбцы отчета
        self.report = []  # сам отчет
        
    def api_request(self, offset=1):
        """Запрос к API Метрики, возвращает словарь с отчетом"""
        params = {
            'id': self.counter,
            'date1': self.start_date,
            'date2': self.end_date,
            'metrics': self.metrics,
            'dimensions': self.dimensions,
            'limit': self.limit,
            'offset': offset,
            'accuracy': 1,
        }
        
        response = requests.get(API_URL, params=params, headers=self.headers)
        return response.json()['data']
    
    def traffic(self):
        """Отчет по посещаемости"""
        self.metrics = 'ym:s:visits'
        self.dimensions = 'ym:s:date'
        
        report = self.full_report()
        self.report = report
        
    def browsers(self):
        """Отчет по браузерам"""
        self.metrics = 'ym:s:visits'
        self.dimensions = 'ym:s:date,ym:s:browser'
        
        report = self.full_report()
        self.report = report
    
    def reformat_api_report(self):
        """Упрощение отчета до списка списков"""
        reformatted_report = []

        for line in self.report:
            dimensions = [x['name'] for x in line['dimensions']]
            reformatted_report.append(dimensions + line['metrics'])

        self.report = reformatted_report
    
    def full_report(self):
        """Постраничная выгрузка из Метрики"""
        full_data = []
        offset = 1
        
        while True:
            print('Starting offset {}'.format(offset))
            data = self.api_request(offset=offset)
            full_data += data
            
            offset += self.limit
            if not data or len(data) < self.limit:
                break
        
        return full_data

In [81]:
m = Metrika(start_date='2023-01-01', end_date='2023-01-14')

In [82]:
m.traffic()

Starting offset 1
Starting offset 6
Starting offset 11


In [83]:
m.report

[{'dimensions': [{'name': '2023-01-11'}], 'metrics': [910.0]},
 {'dimensions': [{'name': '2023-01-12'}], 'metrics': [839.0]},
 {'dimensions': [{'name': '2023-01-10'}], 'metrics': [836.0]},
 {'dimensions': [{'name': '2023-01-09'}], 'metrics': [797.0]},
 {'dimensions': [{'name': '2023-01-13'}], 'metrics': [767.0]},
 {'dimensions': [{'name': '2023-01-05'}], 'metrics': [521.0]},
 {'dimensions': [{'name': '2023-01-04'}], 'metrics': [512.0]},
 {'dimensions': [{'name': '2023-01-03'}], 'metrics': [476.0]},
 {'dimensions': [{'name': '2023-01-14'}], 'metrics': [439.0]},
 {'dimensions': [{'name': '2023-01-06'}], 'metrics': [433.0]},
 {'dimensions': [{'name': '2023-01-07'}], 'metrics': [432.0]},
 {'dimensions': [{'name': '2023-01-08'}], 'metrics': [418.0]},
 {'dimensions': [{'name': '2023-01-02'}], 'metrics': [395.0]},
 {'dimensions': [{'name': '2023-01-01'}], 'metrics': [246.0]}]

In [84]:
m.reformat_api_report()
m.report

[['2023-01-11', 910.0],
 ['2023-01-12', 839.0],
 ['2023-01-10', 836.0],
 ['2023-01-09', 797.0],
 ['2023-01-13', 767.0],
 ['2023-01-05', 521.0],
 ['2023-01-04', 512.0],
 ['2023-01-03', 476.0],
 ['2023-01-14', 439.0],
 ['2023-01-06', 433.0],
 ['2023-01-07', 432.0],
 ['2023-01-08', 418.0],
 ['2023-01-02', 395.0],
 ['2023-01-01', 246.0]]

In [85]:
m.browsers()
m.reformat_api_report()
m.report

Starting offset 1
Starting offset 6
Starting offset 11
Starting offset 16
Starting offset 21
Starting offset 26
Starting offset 31
Starting offset 36
Starting offset 41
Starting offset 46
Starting offset 51
Starting offset 56
Starting offset 61
Starting offset 66
Starting offset 71
Starting offset 76
Starting offset 81
Starting offset 86
Starting offset 91
Starting offset 96
Starting offset 101
Starting offset 106
Starting offset 111
Starting offset 116
Starting offset 121
Starting offset 126
Starting offset 131
Starting offset 136
Starting offset 141
Starting offset 146
Starting offset 151
Starting offset 156
Starting offset 161
Starting offset 166
Starting offset 171
Starting offset 176
Starting offset 181
Starting offset 186
Starting offset 191
Starting offset 196
Starting offset 201
Starting offset 206
Starting offset 211
Starting offset 216
Starting offset 221
Starting offset 226
Starting offset 231
Starting offset 236
Starting offset 241


[['2023-01-11', 'Google Chrome', 374.0],
 ['2023-01-12', 'Google Chrome', 340.0],
 ['2023-01-09', 'Google Chrome', 320.0],
 ['2023-01-10', 'Google Chrome', 319.0],
 ['2023-01-13', 'Google Chrome', 304.0],
 ['2023-01-05', 'Google Chrome', 202.0],
 ['2023-01-04', 'Google Chrome', 183.0],
 ['2023-01-03', 'Google Chrome', 177.0],
 ['2023-01-12', 'Яндекс.Браузер', 162.0],
 ['2023-01-09', 'Chrome Mobile', 160.0],
 ['2023-01-10', 'Chrome Mobile', 157.0],
 ['2023-01-13', 'Chrome Mobile', 157.0],
 ['2023-01-08', 'Chrome Mobile', 154.0],
 ['2023-01-11', 'Яндекс.Браузер', 152.0],
 ['2023-01-04', 'Chrome Mobile', 151.0],
 ['2023-01-07', 'Chrome Mobile', 149.0],
 ['2023-01-14', 'Chrome Mobile', 149.0],
 ['2023-01-05', 'Chrome Mobile', 144.0],
 ['2023-01-10', 'Яндекс.Браузер', 136.0],
 ['2023-01-11', 'Chrome Mobile', 136.0],
 ['2023-01-06', 'Chrome Mobile', 135.0],
 ['2023-01-02', 'Google Chrome', 134.0],
 ['2023-01-03', 'Chrome Mobile', 133.0],
 ['2023-01-01', 'Chrome Mobile', 132.0],
 ['2023-01-06